In [5]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from bs4 import BeautifulSoup

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt

import sddk

# google sheets integration:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [3]:
### configure session and groupurl
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [6]:
# to access our gsheet with data overview & metadata, you need a Google Service Account json file
# (see google-auth python package documentation to start with)
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
try:
    file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
except:
    print("cannot find file ServiceAccountsKey.json")
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
EDH_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/164MLxVcCZg95Bzf9fVyD1-iCA5V97eM3KAFllyhTvt4/edit?usp=sharing")

# Merge the inscriptions with geographies

In [7]:
### upload the raw inscriptions (extracted in another notebook)
### use the line below to upload the old data (obtained on province basis)
### inscriptions_data_df = pd.DataFrame(s.get("https://sciencedata.dk/files/personal_folder/EDH_data/EDH_inscriptions_raw.json").json())
inscriptions_data_df = sddk.read_file("SDAM_data/EDH/EDH_onebyone_2020-09-14.json", "df", conf)
inscriptions_data_df.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,not_before,findspot,year_of_find,present_location,external_image_uris,religion,fotos,geography,social_economic_legal_history,military
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,0071,None,None,None,None,None,None,None,None,None
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",None,...,0051,"Via Nomentana, S. Alessandro, Kirche",1937,None,None,None,None,None,None,None
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,0131,None,before 1975,"Sevilla, Privatbesitz",None,None,None,None,None,None
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,0151,None,before 1979,"Carcabuey, Grupo Escolar",[http://cil-old.bbaw.de/test06/bilder/datenban...,names of pagan deities,None,None,None,None
4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",None,...,0001,Via Cupa (ehem. Vigna Nardi),None,None,None,None,None,None,None,None


In [8]:
# upload geographies
EDH_geo_dict = sddk.read_file("SDAM_data/EDH/EDH_geo_dict_2020-09-15.json", "dict", conf)

In [9]:
EDH_geo_dict["0"]

{'geometry': {'coordinates': [23.451654999, 38.497692999], 'type': 'Point'},
 'properties': {'ancient_findspot': 'Anthedon',
  'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/0',
  'pleiades_uri': 'https://pleiades.stoa.org/places/540639',
  'trismegistos_geo_uri': 'https://www.trismegistos.org/place/15743'},
 'type': 'Feature'}

In [10]:
%%time


### use "edh_geography_uri" column to
### put information from geo_data_df "coordinates" column
### and return it as a new "coordinates" column in the inscriptions_data_df 

def get_coordinates(uri_value):
  try:
    feature = EDH_geo_dict[uri_value.rpartition("/")[2]]
    return feature["geometry"]["coordinates"]
  except:
    return None
                           
inscriptions_data_df["coordinates"]  =  inscriptions_data_df.apply(lambda row: get_coordinates(row["edh_geography_uri"]), axis=1)

CPU times: user 1.02 s, sys: 38.6 ms, total: 1.06 s
Wall time: 1.06 s


In [11]:
len(inscriptions_data_df)

81476

In [12]:
# previously we had 77296
len(inscriptions_data_df[inscriptions_data_df["coordinates"].notnull()]) ### in the previous version (on province basis), we were able to geolocate 69735 from 72483 inscriptions

79012

# Merge with data from XML files

In [13]:
EDH_xml_data = sddk.read_file("SDAM_data/EDH/edh_xml_data_2020-09-22.json", "df", conf)
EDH_xml_data.head(5)

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,HD000002,265631,"[http://www.trismegistos.org/place/000172, htt...",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
1,HD000003,220675,"[http://www.trismegistos.org/place/025443, htt...",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
2,HD000004,222102,"[http://www.trismegistos.org/place/025443, htt...",AVSLLA Marci Porci Nigri serva dominae Veneri ...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
3,HD000005,265629,"[http://www.trismegistos.org/place/000172, htt...",libertus Successus Luci libertus Irenaeus C...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift
4,HD000006,222924,"[http://www.trismegistos.org/place/025443, htt...",Dis Manibus sacrum Memmia Auctina annorum LXX...,71 AD – 150 AD,21,unbestimmt,250,Stele,60,Kalkstein,1000,92,Grabinschrift


In [14]:
# previously we had 78631
len(EDH_xml_data)

81143

In [19]:
EDH_xml_data.rename(columns={"idno_uri":"id"}, inplace=True)

In [20]:
#EDH_xml_data["idno_uri"].replace({r"http:" : r"https:"}, regex=True, inplace=True) ### URIs via API follow HTTPS protocol
EDH_xml_data.head(5)

,id,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,HD000002,265631,"[http://www.trismegistos.org/place/000172, htt...",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
1,HD000003,220675,"[http://www.trismegistos.org/place/025443, htt...",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
2,HD000004,222102,"[http://www.trismegistos.org/place/025443, htt...",AVSLLA Marci Porci Nigri serva dominae Veneri ...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
3,HD000005,265629,"[http://www.trismegistos.org/place/000172, htt...",libertus Successus Luci libertus Irenaeus C...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift
4,HD000006,222924,"[http://www.trismegistos.org/place/025443, htt...",Dis Manibus sacrum Memmia Auctina annorum LXX...,71 AD – 150 AD,21,unbestimmt,250,Stele,60,Kalkstein,1000,92,Grabinschrift


In [21]:
# last time we had (80270, 37)
inscriptions_data_df.shape

(81476, 37)

In [22]:
inscriptions_rich = inscriptions_data_df.merge(EDH_xml_data, on="id", how="left")
inscriptions_rich.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",None,...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",None,...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift


In [23]:
inscriptions_rich.columns

Index(['responsible_individual', 'type_of_inscription', 'letter_size',
       'not_after', 'literature', 'work_status', 'height', 'diplomatic_text',
       'people', 'depth', 'material', 'type_of_monument', 'province_label',
       'width', 'transcription', 'country', 'uri', 'findspot_ancient',
       'last_update', 'modern_region', 'findspot_modern', 'language', 'id',
       'edh_geography_uri', 'commentary', 'trismegistos_uri', 'not_before',
       'findspot', 'year_of_find', 'present_location', 'external_image_uris',
       'religion', 'fotos', 'geography', 'social_economic_legal_history',
       'military', 'coordinates', 'idno_tm', 'placenames_refs', 'text_edition',
       'origdate_text', 'layout_execution', 'layout_execution_text',
       'support_objecttype', 'support_objecttype_text', 'support_material',
       'support_material_text', 'support_decoration', 'keywords_term',
       'keywords_term_text'],
      dtype='object')

In [18]:
# last time we had (80270, 41)
inscriptions_rich.shape

(81476, 52)

In [26]:
EDH_merged_columns = pd.DataFrame(pd.DataFrame(inscriptions_rich).columns, columns=["columns"])

In [27]:
# uncomment to export to gsheets
set_with_dataframe(EDH_overview.add_worksheet("EDH_merged_columns", 1, 1), EDH_merged_columns)

# Upload the merged data to sciencedata.dk

In [28]:
%%time
### put your dataframe data into this folder

sddk.write_file("SDAM_data/EDH/EDH_merged_2020-09-22.json", inscriptions_rich, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/EDH_merged_2020-09-22.json"
CPU times: user 1.94 s, sys: 863 ms, total: 2.8 s
Wall time: 7.22 s


# Reading the file back and exporting a feather file (UNDER DEVELOPMENT)

In [4]:
inscriptions_rich = sddk.read_file("SDAM_data/EDH/EDH_merged_2020-09-22.json", "df", conf)

In [5]:
inscriptions_rich.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,support_decoration_text,keywords_term,keywords_term_text
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,None,None,None,None,None,None,None,None,None,None
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",None,...,"{'type': 'execution', 'ref': 'http://www.eagle...",unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/257,Tafel,http://www.eagle-network.eu/voc/material/lod/48,Marmor,"{'type': 'decoration', 'ref': 'http://www.eagl...",nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,"{'type': 'execution', 'ref': 'http://www.eagle...",unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/57,Statuenbasis,http://www.eagle-network.eu/voc/material/lod/48,Marmor,"{'type': 'decoration', 'ref': 'http://www.eagl...",nein,http://www.eagle-network.eu/voc/typeins/lod/69,Ehreninschrift
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,"{'type': 'execution', 'ref': 'http://www.eagle...",unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/29,Altar,http://www.eagle-network.eu/voc/material/lod/60,Kalkstein,"{'type': 'decoration', 'ref': 'http://www.eagl...",nein,http://www.eagle-network.eu/voc/typeins/lod/80,Weihinschrift
4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",None,...,"{'type': 'execution', 'ref': 'http://www.eagle...",unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/250,Stele,http://www.eagle-network.eu/voc/material/lod/138,unbestimmt,"{'type': 'decoration', 'ref': 'http://www.eagl...",nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift


In [8]:
for column in inscriptions_rich.columns: # to avoid problems with encoding, lets check that everything is utf-8
    try: 
        inscriptions_rich[column] = inscriptions_rich[column].str.encode("utf-8")
    except: 
        inscriptions_rich[column] = inscriptions_rich[column]  

In [12]:
from pyarrow import feather

In [20]:
pyarrow.feather.write_feather(inscriptions_rich.reset_index(), "test-utf8.feather", version=1, compression=None)

In [21]:
conf[0].put(conf[1] + "SDAM_data/EDH/test-utf8.feather", data=open("test-utf8.feather", "rb"))

<Response [201]>

In [17]:
EDH_feather = pd.read_feather("test-utf8.feather")
for column in EDH_feather.columns:
    try:
        EDH_feather[column] = EDH_feather[column].str.decode("utf-8")
    except:
        EDH_feather[column] = EDH_feather[column]
EDH_feather.head()

,index,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,...,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,support_decoration_text,keywords_term,keywords_term_text
0,NaN,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,NaN,...,NaN,None,None,None,None,None,NaN,None,None,None
1,NaN,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/257,Tafel,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
2,NaN,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/57,Statuenbasis,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/69,Ehreninschrift
3,NaN,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/29,Altar,http://www.eagle-network.eu/voc/material/lod/60,Kalkstein,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/80,Weihinschrift
4,NaN,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/250,Stele,http://www.eagle-network.eu/voc/material/lod/138,unbestimmt,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift


In [15]:
sddk.write_file("SDAM_data/EDH/EDH_merged_2020-09-18.feather", inscriptions_rich.reset_index(), conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/EDH_merged_2020-09-18.feather"


In [16]:
EDH_feather = sddk.read_file("SDAM_data/EDH/EDH_merged_2020-09-18.feather", "df", conf)

In [17]:
EDH_feather.head(5)

,index,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,...,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,support_decoration_text,keywords_term,keywords_term_text
0,0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,NaN,...,NaN,None,None,None,None,None,NaN,None,None,None
1,1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/257,Tafel,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
2,2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/57,Statuenbasis,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/69,Ehreninschrift
3,3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/29,Altar,http://www.eagle-network.eu/voc/material/lod/60,Kalkstein,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/80,Weihinschrift
4,4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/250,Stele,http://www.eagle-network.eu/voc/material/lod/138,unbestimmt,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
